# Object Creation: Self-Report Stress Objects from Mood Events

This notebook demonstrates how to create stress self-report objects from mood events in the OCED data. Each stress self-report object represents a stress value reported in a mood event, and is linked to:
<ul>
<li>The mood event that reported the stress
<li>The day on which it occurred
<li>The user who reported it
<li>The nearest notification within a specified time period (if any exists)

## 1. Setup

In [1]:

import sys
sys.path.append('..')
from pathlib import Path
import pandas as pd
from datetime import timedelta
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.stress_objects import StressObjectManager

## 2. Load OCED Data

In [2]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"notifications_processed.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

# Diagnostic code to inspect notification events
mood_events = [
    notif for notif in data_dict.get('behaviorEvents', [])
    if notif['behaviorEventType'] == 'mood'
]
print(len(mood_events))
mood_events

Number of behavior events: 134
Number of objects: 75
60


[{'id': 'ffe82200-553a-4a32-b11c-dd47aa4bf74b',
  'behaviorEventType': 'mood',
  'time': '2025-05-21T08:50:42.042000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 5},
   {'name': 'arousal', 'value': 3},
   {'name': 'stress', 'value': 6}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id': 'd40bf9fa-b90d-4fa6-96a6-7eb031154e3f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_type': 'behavior'},
 {'id': '891913ae-361e-4fdb-ac13-3fa459b4f278',
  'behaviorEventType': 'mood',
  'time': '2025-05-18T17:09:34.215000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 6}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '43cb2837-59d9-4394-a3c6-5b707b73040f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_

## 3. Initialize Stress Object Manager
Create an instance of the StressObjectManager class and add the stress self-report object type to the data.

In [3]:
# Initialize the stress object manager
manager = StressObjectManager()

# Add stress self-report object type to the data
data = manager.create_stress_object_type(data_dict)

# Print object types to verify
print("Object types in data:")
for obj_type in data.get('objectTypes', []):
    print(f"- {obj_type['name']}")

Stress self-report object type added.
Object types in data:
- player
- intervention
- day
- week
- physical_activity_bout
- notification
- stress_self_report


## 4. Create Stress Self-Report Objects
Now we'll create stress self-report objects from the mood events. We'll use a 1-hour time window to look for nearby notifications.

In [4]:
# Get the user ID from the data
user_objects = [obj for obj in data.get('objects', []) if obj['type'] == 'player']
if not user_objects:
    raise ValueError("No user object found in the data")
user_id = user_objects[0]['id']
print(f"Using user ID: {user_id}")

# Create stress self-report objects with a 1-hour time window
time_period = timedelta(hours=5)
extended_data, stress_objects = manager.create_stress_objects(
    data=data,
    user_id=user_id,
    time_period=time_period
)

Using user ID: e4365330-3468-48c3-bce3-145668e06f8f

Found 60 mood events
Found 0 existing stress self-report objects


Processing mood events:  13%|█▎        | 8/60 [00:00<00:00, 76.65it/s]

Creating new stress self-report object b67ff5fd-acf7-4f79-a8cc-4753f752e732 for mood event
Creating new stress self-report object d4cb794c-095d-477c-836b-c54b8594db74 for mood event
Creating new stress self-report object 036e2ed8-739a-4953-be3d-5fe729b0819f for mood event
Creating new stress self-report object 6a6cea14-fbf3-4983-bf1b-5e854e7b1d06 for mood event
Creating new stress self-report object 663d5d6d-d993-4194-9e5d-f948fa573dd1 for mood event
Creating new stress self-report object 1923fafb-849e-44c5-8720-a137f4e0c259 for mood event
Creating new stress self-report object 447aca72-b8a8-4ae6-85f5-ef6a7f5ee7fc for mood event
Creating new stress self-report object 99d80948-8cd8-4f16-a9f0-6cf31f08d2d9 for mood event
Creating new stress self-report object 5f7b7fb5-a2e2-4883-9966-fc3336406183 for mood event
Creating new stress self-report object 68250fd7-aebf-4665-bf69-d451eb3abe71 for mood event
Creating new stress self-report object 8aef79a4-ba09-4ad9-81c3-62142bcf103d for mood event

Processing mood events:  40%|████      | 24/60 [00:00<00:00, 71.18it/s]

Creating new stress self-report object 837acee3-677d-4f1a-b1bd-a428d56e4520 for mood event
Creating new stress self-report object e0bc8549-b8e5-4532-b112-1c144c54215b for mood event
Creating new stress self-report object 43ece059-0428-4dee-b1be-b5be4c3955e3 for mood event
Creating new stress self-report object 2d92447f-e445-433e-a7e4-b0ad01ba575b for mood event
Creating new stress self-report object 8706cafb-f89b-4242-ac8f-e26fcb9e16b8 for mood event
Creating new stress self-report object 3a3e9928-f221-48c9-ac54-9ec77f6490d7 for mood event
Creating new stress self-report object 76ac4ac1-1114-4d29-8d21-eaf4d1fb6277 for mood event
Creating new stress self-report object 6aa72906-eb4f-49b9-ad21-3ed4a183fef8 for mood event
Creating new stress self-report object e77de6ab-3d2c-4276-8919-df2bd29f9d7b for mood event
Creating new stress self-report object bf66a170-b236-4e7b-8886-f870c91747ed for mood event
Creating new stress self-report object dd63f4e3-9299-467e-b12c-4f1cab533337 for mood event

...

Linking stress self-report object to nearby notification 1eb516b7-d559-49f3-bffe-69ed5e809c75
Creating new stress self-report object a981a7f1-fd41-41fc-9d2c-5c5b0ab3fe08 for mood event
Linking stress self-report object to nearby notification aefa1a47-b545-482c-80ba-043be1117447
Creating new stress self-report object 10428b0c-42fb-4c3e-b502-006e69c935ce for mood event
Linking stress self-report object to nearby notification 689d1c91-19bf-4981-8759-a499778dfda4
Creating new stress self-report object 9532eeec-9250-4852-91a6-dc0f30734b1f for mood event
Linking stress self-report object to nearby notification 8a0ecac3-dcfe-4c61-b072-2614d8067954
Creating new stress self-report object 0a0d548b-e19b-4702-b4bf-ccd0f0c38374 for mood event
Linking stress self-report object to nearby notification 8a0ecac3-dcfe-4c61-b072-2614d8067954
Creating new stress self-report object 84084c63-72c1-4464-8773-955618981eb5 for mood event
Linking stress self-report object to nearby notification 8b1192ec-01fc-4bb4

Processing mood events:  88%|████████▊ | 53/60 [00:04<00:00,  7.70it/s]

Linking stress self-report object to nearby notification 9d24ec6f-0c5a-422e-bcad-6901e64db073
Creating new stress self-report object 827fc62c-41b9-415a-ae35-cc8e1c4fa5c0 for mood event
Linking stress self-report object to nearby notification 0b610bad-b3f1-49b0-94ac-fe98efcfbfcd
Creating new stress self-report object 2e6cbe11-4658-4c85-b908-cd5520f76af7 for mood event
Linking stress self-report object to nearby notification e8922309-a8e3-4007-927d-319269411e18
Creating new stress self-report object d8b52fb6-bcad-433e-8ff5-0b9e2156a468 for mood event
Linking stress self-report object to nearby notification abc75c62-f4e4-4491-a845-a09e5aea65ea
Creating new stress self-report object 3641b7f6-89ef-430b-bb4f-f67dd51dfd9c for mood event
Linking stress self-report object to nearby notification 9cb411ba-c7fc-4407-b19b-bf59adbe8399
Creating new stress self-report object a03b5c50-ef78-44bc-8dd0-5e05844d1a48 for mood event
Linking stress self-report object to nearby notification ce1a21a5-9444-4eb4

Processing mood events: 100%|██████████| 60/60 [00:04<00:00, 13.61it/s]

Creating new stress self-report object a1616fac-3b6c-4b7a-a83c-8d25edd690f7 for mood event
Creating new stress self-report object bf4e06e5-98ac-49cf-abb0-91ce8d238edb for mood event

Processing complete:
- Created 60 new stress self-report objects
- Total stress self-report objects: 60


## 5. Analyze Created Objects
Let's analyze the created stress self-report objects and their relationships.

In [5]:
# Count stress self-report objects with notification relationships
stress_with_notifications = [
    obj for obj in stress_objects
    if any(rel['qualifier'] == 'follows_notification' for rel in obj.get('relationships', []))
]

print(f"Stress self-report objects analysis:")
print(f"- Total stress self-report objects: {len(stress_objects)}")
print(f"- Stress self-report objects with notification relationships: {len(stress_with_notifications)}")
print(f"- Percentage with notifications: {len(stress_with_notifications)/len(stress_objects)*100:.1f}%")

# Look at a sample stress self-report object
if stress_objects:
    sample = stress_objects[0]
    print("\nSample stress self-report object:")
    print(f"ID: {sample['id']}")
    print("Attributes:")
    for attr in sample['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in sample.get('relationships', []):
        print(f"- {rel['qualifier']} -> {rel['id']}")

Stress self-report objects analysis:
- Total stress self-report objects: 60
- Stress self-report objects with notification relationships: 25
- Percentage with notifications: 41.7%

Sample stress self-report object:
ID: b67ff5fd-acf7-4f79-a8cc-4753f752e732
Attributes:
- stress_value: 6.0 (at 2025-03-20T15:39:28.698000)

Relationships:
- occurred_on -> 625643ae-4c4c-4e3f-bb41-b84285ccd7e8
- reported_by -> e4365330-3468-48c3-bce3-145668e06f8f


## 6. Save Extended Data
Finally, let's save the extended data with the new stress self-report objects.

In [6]:
# Save the extended data
manager.save_extended_data("stress_self_reports.json", extended_data)

print("\nData saved successfully!")


Saving extended data with:
- 60 stress self-report objects
- 60 mood events
- 135 total objects
- 134 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\stress_self_reports.json

Data saved successfully!


In [7]:
# Count stress self-report objects with notification relationships
data_objects = [
    obj for obj in extended_data.get('objects', [])
    if obj['type'] == 'stress_self_report'
]

print(len(data_objects))
data_objects

60


[{'id': 'b67ff5fd-acf7-4f79-a8cc-4753f752e732',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 6.0,
    'time': '2025-03-20T15:39:28.698000'}],
  'relationships': [{'id': '625643ae-4c4c-4e3f-bb41-b84285ccd7e8',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': 'd4cb794c-095d-477c-836b-c54b8594db74',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-21T11:12:32.086000'}],
  'relationships': [{'id': '5cb0dad5-1a67-48ef-b672-683595141a53',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': '036e2ed8-739a-4953-be3d-5fe729b0819f',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-24T12:16:29.4

In [8]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('objectTypes', [])
]

print(len(data))
data

7


[{'name': 'player', 'attributes': [{'name': 'id', 'type': 'string'}]},
 {'name': 'intervention',
  'attributes': [{'name': 'goal', 'type': 'string'},
   {'name': 'start_date', 'type': 'string'},
   {'name': 'end_date', 'type': 'string'}]},
 {'name': 'day',
  'attributes': [{'name': 'date', 'type': 'string'},
   {'name': 'day_of_week', 'type': 'string'}]},
 {'name': 'week',
  'attributes': [{'name': 'week_start_date', 'type': 'string'},
   {'name': 'week_number', 'type': 'integer'},
   {'name': 'year', 'type': 'integer'}]},
 {'name': 'physical_activity_bout',
  'attributes': [{'name': 'bout_type', 'type': 'string'}]},
 {'name': 'notification',
  'attributes': [{'name': 'last_action', 'type': 'string'}]},
 {'name': 'stress_self_report',
  'attributes': [{'name': 'stress_value', 'type': 'number'}]}]

In [10]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('behaviorEvents', [])
    if obj['behaviorEventType'] == 'mood'
]

print(len(data))
data

60


[{'id': 'ffe82200-553a-4a32-b11c-dd47aa4bf74b',
  'behaviorEventType': 'mood',
  'time': '2025-05-21T08:50:42.042000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 5},
   {'name': 'arousal', 'value': 3},
   {'name': 'stress', 'value': 6}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id': 'd40bf9fa-b90d-4fa6-96a6-7eb031154e3f',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'bf4e06e5-98ac-49cf-abb0-91ce8d238edb',
    'type': 'object',
    'qualifier': 'reports_stress'}],
  'event_type': 'behavior'},
 {'id': '891913ae-361e-4fdb-ac13-3fa459b4f278',
  'behaviorEventType': 'mood',
  'time': '2025-05-18T17:09:34.215000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 6}],
  'relationships': [{'id': 'e4365330-3468-48c3-bce3-145668e06f8f',
    'type': 'object',
    'qualifier': 'source'},
   {'id